[From this Kaggle course](https://www.kaggle.com/code/alexisbcook/n-step-lookahead)

## Previous agents:

In [49]:
import random
import numpy as np
from kaggle_environments import make, evaluate

In [50]:
def score_move_NoLook(grid, col, mark, config):
    next_grid = drop_piece_NoLook(grid, col, mark, config)
    score = get_heuristic_NoLook(next_grid, mark, config)
    return score

def drop_piece_NoLook(grid, col, mark, config):
    next_grid = grid.copy()
    for row in range(config.rows-1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = mark
    return next_grid

def get_heuristic_NoLook(grid, mark, config):
    num_threes = count_windows(grid, 3, mark, config)
    num_fours = count_windows(grid, 4, mark, config)
    num_threes_opp = count_windows(grid, 3, mark%2+1, config)
    score = num_threes - 1e2*num_threes_opp + 1e6*num_fours
    return score

def check_window(window, num_discs, piece, config):
    return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)
    
def count_windows(grid, num_discs, piece, config):
    num_windows = 0
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    return num_windows

def NoLookAheadAgent(obs, config, get_scores = False):
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    scores = dict(zip(valid_moves, [score_move_NoLook(grid, col, obs.mark, config) for col in valid_moves]))
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    if get_scores:
        return random.choice(max_cols), scores
    return random.choice(max_cols)

In [63]:
def OneStepLookAheadAgent(obs, config, get_scores = False):
    
    import numpy as np
    import random

    A = 100
    B = 5
    C = 1
    D = -2
    E = -20

    def score_move_OneStep(grid, col, mark, config):
        next_grid = drop_piece_OneStep(grid, col, mark, config)
        score = get_heuristic_OneStep(next_grid, mark, config)
        return score

    def drop_piece_OneStep(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    def get_heuristic_OneStep(grid, mark, config):
        num_twos = count_windows_OneStep(grid, 2, mark, config)
        num_threes = count_windows_OneStep(grid, 3, mark, config)
        num_fours = count_windows_OneStep(grid, 4, mark, config)
        num_twos_opp = count_windows_OneStep(grid, 2, mark%2+1, config)
        num_threes_opp = count_windows_OneStep(grid, 3, mark%2+1, config)
        score = A*num_fours + B*num_threes + C*num_twos + D*num_twos_opp + E*num_threes_opp
        print(grid)
        print(num_twos, num_threes, num_fours, num_twos_opp, num_threes_opp, score)
        print("______")
        return score

    def check_window_OneStep(window, num_discs, piece, config):
        return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)
        
    def count_windows_OneStep(grid, num_discs, piece, config):
        num_windows = 0
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                if check_window_OneStep(window, num_discs, piece, config):
                    num_windows += 1
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if check_window_OneStep(window, num_discs, piece, config):
                    num_windows += 1
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if check_window_OneStep(window, num_discs, piece, config):
                    num_windows += 1
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if check_window_OneStep(window, num_discs, piece, config):
                    num_windows += 1
        return num_windows
    
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    scores = dict(zip(valid_moves, [score_move_OneStep(grid, col, obs.mark, config) for col in valid_moves]))
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    if get_scores:
        return random.choice(max_cols), scores, max_cols
    return random.choice(max_cols)

## Scores

In [64]:
from tqdm import tqdm

def get_win_percentages(agent1, agent2, n_rounds=100, info=False):
    config = {'rows': 6, 'columns': 7, 'inarow': 4}  
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)  
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    agent_1_procent = np.round(outcomes.count([1,-1])/len(outcomes)*100, 2)
    agent_2_procent = np.round(outcomes.count([-1,1])/len(outcomes)*100, 2)
    if info:
        print("Agent 1 Win Percentage:", agent_1_procent, "%")
        print("Agent 2 Win Percentage:", agent_2_procent, "%")
        print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
        print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))
    return (agent_1_procent, agent_2_procent)

def get_percentage_table(agents, names, n_rounds=20):
    table = []
    header = ["Agent"] + names
    table.append(header)

    for j in tqdm(range(len(agents))):
        agent1 = agents[j]
        row = [names[j]]
        for i in range(len(agents)):
            agent2 = agents[i]
            if agent1 == agent2:
                row.append("-")
                continue
            agent1_percentage, agent2_percentage = get_win_percentages(agent1, agent2, n_rounds=n_rounds)
            print(agent1_percentage, agent2_percentage)
            row.append(f"{agent1_percentage:.2f}% over {names[i]}")
        table.append(row)

    max_lengths = [max(len(str(row[i])) for row in table) for i in range(len(header))]

    for row in table:
        print(" | ".join(str(row[i]).ljust(max_lengths[i]) for i in range(len(header))))

## Minimax algorithm (New agent)

In [65]:
def drop_piece(grid, col, mark, config):
    next_grid = grid.copy()
    for row in range(config.rows-1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = mark
    return next_grid

def check_window(window, num_discs, piece, config):
    return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)

def count_windows(grid, num_discs, piece, config):
    num_windows = 0
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    return num_windows

In [66]:
def get_heuristic(grid, mark, config):
    num_threes = count_windows(grid, 3, mark, config)
    num_fours = count_windows(grid, 4, mark, config)
    num_threes_opp = count_windows(grid, 3, mark%2+1, config)
    num_fours_opp = count_windows(grid, 4, mark%2+1, config)
    score = num_threes - 1e2*num_threes_opp - 1e4*num_fours_opp + 1e6*num_fours
    return score

In [67]:
def score_move(grid, col, mark, config, nsteps):
    next_grid = drop_piece(grid, col, mark, config)
    score = minimax(next_grid, nsteps-1, False, mark, config)
    return score

def is_terminal_window(window, config):
    return window.count(1) == config.inarow or window.count(2) == config.inarow

def is_terminal_node(grid, config):
    if list(grid[0, :]).count(0) == 0:
        return True 
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if is_terminal_window(window, config):
                return True
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if is_terminal_window(window, config):
                return True
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if is_terminal_window(window, config):
                return True
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if is_terminal_window(window, config):
                return True
    return False

def minimax(node, depth, maximizingPlayer, mark, config):
    is_terminal = is_terminal_node(node, config)
    valid_moves = [c for c in range(config.columns) if node[0][c] == 0]
    if depth == 0 or is_terminal:
        return get_heuristic(node, mark, config)
    if maximizingPlayer:
        value = -np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, mark, config)
            value = max(value, minimax(child, depth-1, False, mark, config))
        return value
    else:
        value = np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, mark%2+1, config)
            value = min(value, minimax(child, depth-1, True, mark, config))
        return value

In [68]:
N_STEPS = 3

def nStepLookAheadAgent(obs, config):
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    return random.choice(max_cols)

In [58]:
get_win_percentages(agent1=nStepLookAheadAgent, agent2="random", n_rounds=10)

(100.0, 0.0)

In [72]:
get_win_percentages(agent1=nStepLookAheadAgent, agent2=NoLookAheadAgent, n_rounds=10, info=True)

Agent 1 Win Percentage: 90.0 %
Agent 2 Win Percentage: 10.0 %
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0


(90.0, 10.0)

In [75]:
# To get true percentage between two latest agents:
get_win_percentages(agent1=nStepLookAheadAgent, agent2=OneStepLookAheadAgent, n_rounds=100, info=True)

Agent 1 Win Percentage: 49.0 %
Agent 2 Win Percentage: 45.0 %
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0


(49.0, 45.0)

In [76]:
get_win_percentages(agent1=NoLookAheadAgent, agent2=OneStepLookAheadAgent, n_rounds=100, info=True)

Agent 1 Win Percentage: 23.0 %
Agent 2 Win Percentage: 77.0 %
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0


(23.0, 77.0)

In [77]:
get_win_percentages(agent1=nStepLookAheadAgent, agent2=NoLookAheadAgent, n_rounds=100, info=True)

Agent 1 Win Percentage: 78.0 %
Agent 2 Win Percentage: 19.0 %
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0


(78.0, 19.0)

#### Table

In [70]:
agents = ["random", NoLookAheadAgent, OneStepLookAheadAgent, nStepLookAheadAgent]
get_percentage_table(agents, names = ["Random", "No lookahead", "One-Step", "3-Step"], n_rounds=20)

  0%|          | 0/4 [00:00<?, ?it/s]

0.0 100.0
0.0 100.0


 25%|██▌       | 1/4 [00:51<02:33, 51.05s/it]

0.0 100.0
100.0 0.0
15.0 85.0


 50%|█████     | 2/4 [01:59<02:02, 61.20s/it]

15.0 85.0
100.0 0.0
65.0 35.0


 75%|███████▌  | 3/4 [03:08<01:04, 64.98s/it]

40.0 60.0
100.0 0.0
80.0 15.0


100%|██████████| 4/4 [06:00<00:00, 90.05s/it] 

40.0 50.0
Agent        | Random              | No lookahead             | One-Step             | 3-Step            
Random       | -                   | 0.00% over No lookahead  | 0.00% over One-Step  | 0.00% over 3-Step 
No lookahead | 100.00% over Random | -                        | 15.00% over One-Step | 15.00% over 3-Step
One-Step     | 100.00% over Random | 65.00% over No lookahead | -                    | 40.00% over 3-Step
3-Step       | 100.00% over Random | 80.00% over No lookahead | 40.00% over One-Step | -                 


### Due to slow processing times, the number of games played is only 10, which makes to percentages extremely unreliable